In [1]:
import numpy as np
import subprocess

%run ../potencia-deflacion/met-potencia.ipynb # me importo eigen y power_iteration
%run ../datos/datos.ipynb # me importo dataset

In [2]:
X_train, y_train, _, _ = datosConDistribucion(1000)

In [3]:
def centralizar(A):
    u = np.mean(A, axis=0)
    X = A - u
    return X

In [4]:
def pca(A):
    subprocess.run([
    'g++', '-o', f'./covarianza-c/met_potencia', '../potencia-deflacion/met_potencia.cpp',
    '-std=c++11', '-I', 'Eigen', '-O3'
    ])

    X = centralizar(A)

    #matriz de covarianza
    cov_matriz =  (1 / (A.shape[0] - 1)) * np.dot(X.T,X)
    
    !rm ./covarianza-c/matriz_cov_entrada.txt
    with open(f'./covarianza-c/matriz_cov_entrada.txt','a') as f:
        f.write(f"{cov_matriz.shape[0]} {cov_matriz.shape[1]}\n") #Tamaño de la matriz
        np.savetxt(f, cov_matriz, newline="\n")

    result = subprocess.run([
        "./covarianza-c/met_potencia", f"./covarianza-c/matriz_cov_entrada.txt"
    ], stdout=subprocess.PIPE)

    !rm ./covarianza-c/matriz_cov_salida.txt
    with open(f'./covarianza-c/matriz_cov_salida.txt', "a") as output:
        output.write(result.stdout.decode('utf-8'))
    
    arr = np.loadtxt(f'./covarianza-c/matriz_cov_salida.txt')
    
    eigenvalues = arr[1] # autovalores
    eigenvectors = np.delete(arr, [0, 1], axis=0) #autovectores como columnas


    sorted_indices = np.argsort(eigenvalues)[::-1]
    autovalores_ord = eigenvalues[sorted_indices]
    autovectores_ord = eigenvectors[:, sorted_indices]

    !rm ./covarianza-c/autovectores_cov.txt
    with open(f'../pca/covarianza-c/autovectores_cov.txt','a') as f:
        np.savetxt(f, autovectores_ord, newline="\n")  

    !rm ./covarianza-c/autovalores_cov.txt
    with open(f'../pca/covarianza-c/autovalores_cov.txt','a') as f:
        np.savetxt(f, autovalores_ord)   

    return autovalores_ord, autovectores_ord

In [5]:
pca(X_train)

(array([ 5.72219e+01,  2.34710e+01,  1.36615e+01,  8.59512e+00,
         8.27344e+00,  6.57182e+00,  5.84917e+00,  5.54496e+00,
         5.13560e+00,  4.97187e+00,  4.68290e+00,  4.47837e+00,
         4.20212e+00,  4.06896e+00,  3.97621e+00,  3.69818e+00,
         3.55433e+00,  3.44949e+00,  3.35807e+00,  3.31715e+00,
         3.14013e+00,  3.08322e+00,  2.99824e+00,  2.96911e+00,
         2.90666e+00,  2.77943e+00,  2.69780e+00,  2.62523e+00,
         2.56581e+00,  2.49313e+00,  2.45942e+00,  2.36186e+00,
         2.33036e+00,  2.28063e+00,  2.18523e+00,  2.14750e+00,
         2.06291e+00,  2.03620e+00,  2.00717e+00,  1.96543e+00,
         1.93210e+00,  1.91379e+00,  1.82999e+00,  1.79080e+00,
         1.77053e+00,  1.73248e+00,  1.71363e+00,  1.67981e+00,
         1.66575e+00,  1.63330e+00,  1.57700e+00,  1.54933e+00,
         1.51821e+00,  1.50686e+00,  1.49243e+00,  1.46011e+00,
         1.42758e+00,  1.40449e+00,  1.37781e+00,  1.37383e+00,
         1.34173e+00,  1.30583e+00,  1.2

In [6]:
autovalores_ord = np.loadtxt(f'../pca/covarianza-c/autovalores_cov.txt')
autovectores_ord = np.loadtxt(f'../pca/covarianza-c/autovectores_cov.txt')

In [7]:
#Calculo la varianza explicada acumulada
var_exp_acum = np.cumsum(autovalores_ord) / np.sum(autovalores_ord)
#print(var_exp_acum)
#Encontrar el numero de componentes principales seleccionados mas del 95% varianza acumulada
p_sel = np.argmax(var_exp_acum >= 0.95)

# Selecciona solo las primeras p_sel componentes principales
comp_prin_sel = autovectores_ord[:, :p_sel]

#Nuevo conjunto reducido
x_train_red = np.dot(X_train, comp_prin_sel)
print(p_sel)

147
